# Introdução ao Reconhecimento de Padrões, 2020.2, UFC/DETI
## Trabalho 3

Aluno : Thyago Freitas da Silva <br>
Matrícula : 392035

In [2]:
import numpy as np
import operator
import collections
import pandas
from random import randrange
from sklearn.metrics import confusion_matrix as cm

### Implementacão do classificador "K-Means"

In [102]:
norm_p = lambda x,y,p : abs((x-y))**p
euclidian = lambda list1,list2: np.sqrt(sum(map(norm_p,list1,list2,[2]*len(list1))))

class KMeansClassifier:
    def __init__(self, K, R):
        self.K = K
        self.R = R
        self.centroids = []
    def calc_mean(self,data):
        return np.mean(np.array(data),axis=0).tolist()
    def fit_predict(self, x_train):
        n = x_train.shape[0]
        result = [0]*n
        aux = 0
        stop = False
        idx = np.random.choice(len(x_train), self.K, replace=False)
        for i in idx:
            self.centroids.append(x_train.iloc[i,:].to_list())
        while aux < self.R or stop:
            aux = aux + 1
            for row in range(n):
                d = []
                for c in range(self.K):
                    d.append(euclidian(x_train.iloc[row,:].to_list(),self.centroids[c]))
                result[row] = (d.index(min(d)))
            changes = self.K
            for c_n, c in enumerate(self.centroids):
                values = []
                values.append(c)
                for index, r in enumerate(result):
                    if r == c_n:
                        values.append(x_train.iloc[index,:].to_list())
                new_c = self.calc_mean(values)
                if collections.Counter(new_c) == collections.Counter(c):
                    changes = changes - 1
                self.centroids[c_n] = new_c
            if changes == 0:
                stop = True
        return result

### Leitura da base "demartology"

In [86]:
data = pandas.read_csv("./data/dermatology.csv", header=None)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       366 non-null    int64 
 1   1       366 non-null    int64 
 2   2       366 non-null    int64 
 3   3       366 non-null    int64 
 4   4       366 non-null    int64 
 5   5       366 non-null    int64 
 6   6       366 non-null    int64 
 7   7       366 non-null    int64 
 8   8       366 non-null    int64 
 9   9       366 non-null    int64 
 10  10      366 non-null    int64 
 11  11      366 non-null    int64 
 12  12      366 non-null    int64 
 13  13      366 non-null    int64 
 14  14      366 non-null    int64 
 15  15      366 non-null    int64 
 16  16      366 non-null    int64 
 17  17      366 non-null    int64 
 18  18      366 non-null    int64 
 19  19      366 non-null    int64 
 20  20      366 non-null    int64 
 21  21      366 non-null    int64 
 22  22      366 non-null    in

#### Pre-processamento

Olhando o conteúdo do arquivo, foi notado que algumas linhas da coluna 33 (band-like infiltrate) apresentam valores marcados com "?" que atrapalham o resultado do algoritmo. Para remediar esse problema , decidi trocar os valores faltantes demarcados por "?" pela mediana da coluna 33, pois tal medida é menos sensível a outliers se comparado com a média, por exemplo.

In [87]:
def calc_median(array):
    values = []
    for v in array:
        if v != '?':
            v = int(v)
            values.append(v)
    return np.median(values)

median = calc_median(data.iloc[:,33])
data.iloc[:,33] = list(map(lambda value: median if value == '?' else value, data.iloc[:,33]))
data.iloc[:,33] = data.iloc[:,33].astype(np.int64)

#### Normalizar dados

In [6]:
def normalize(data):
    columns = data.shape[1] - 1 #subtrai 1 para descartar coluna com classes
    for c in range(columns):
        min_v = np.min(data.iloc[:,c])
        max_v = np.max(data.iloc[:,c])
        data.iloc[:,c] = data.iloc[:,c].apply(lambda v : (v - min_v)/(max_v - min_v))
    return data

data = normalize(data)

#### Separar dados em atributos e classes de saída

In [88]:
atributes = data.iloc[:,:34]
target = data.iloc[:,34]

In [101]:
kmeans = KMeansClassifier(5,5)
predict = kmeans.fit_predict(atributes)
len(kmeans.centroids)

366


5